# Experiment 2: Selection Bias and Batch Effects

In this experiment, we combine selection bias and batch effects. The AIRR data comes from 2 hospitals: `hospital1` that recruits mostly diseased and `hospital2` that recruits mostly healthy individuals. We explore two possible scenarios here:

1. the performance of the ML model when there is a selection bias during training, but not in the test dataset.

2. the performance of the ML model when there is no connection between the immune state and AIRR: we will show that ML models will in this case learn only a spurious correlation.

Immune state is a binary variable and can have values `True` or `False` to indicate if an individual is diseased or healthy. AIRR is a set of sequences simulated based on the values of the immune state and the confounder for the given individual. Hospital is a binary variable (`hospital1` or `hospital2`). Each hospital has their own experimental protocol that influences the observed AIRR. The influence of the experimental protocol on AIRR is manifested via higher frequency of some k-mers in the sequenced AIRRs.

Steps for each scenario:

1. Simulate training and test dataset from a causal graph to include the variables as described above.

2. Train an ML model (here: logistic regression on repertoires represented by the k-mer frequencies) on the train set which has selection bias and assess its performance on the test set when there is no selection bias.

Software used: 

- DagSim for simulation of the causal graph; 
- immuneML v2.1 for implanting signal in AIRRs and for training and assessing machine learning classifiers; 
- OLGA for simulation of naive AIRRs

In [1]:
import yaml
import dagsim.base as ds
from util.dataset_util import make_AIRR_dataset, make_dataset, setup_path
from util.experiment2 import make_immune_state_signal
from util.simulation import get_immune_state, get_hospital, get_exp_protocol, get_repertoire, get_selection, get_gen


In [70]:
setup_path("./experiment2") # remove results from the previous run

train_example_count = 200
test_example_count = 50

Removing experiment2...


## Scenario 1: immune state signal is stronger than the influence of the experimental protocol

In [71]:
# define and build path, remove content if not empty

scenario1_path = setup_path("./experiment2/scenario1/")
scenario1_data_path = setup_path(scenario1_path / "data")

### Step 1: AIRR simulation from a causal graph

In [72]:
# define constants for the simulation

p_immune_state = 0.5 # parameter of binomial distribution for the immune state
p_hospital = 0.5 # parameter of binomial distribution for selecting between hospitals 1 and 2

sequence_count = 2000
immune_state_implanting_rate = 0.0015
protocol_implanting_rate = 0.01

immune_state_signal = make_immune_state_signal()


# define nodes of the causal graph

immune_state_node = ds.Node(name="immune_state", function=get_immune_state, kwargs={"p": p_immune_state})

hospital_node = ds.Node(name="hospital", function=get_hospital, kwargs={"p": p_hospital})

experimental_protocol_node = ds.Node(name="exp_protocol", function=get_exp_protocol, kwargs={"hospital": hospital_node})

repertoire_node = ds.Node(name="repertoire", function=get_repertoire,
                             kwargs={"immune_state": immune_state_node,
                                        "experimental_protocol_id": experimental_protocol_node,
                                        "path": scenario1_data_path / "train", "sequence_count": sequence_count, 
                                        "immune_state_signal": immune_state_signal, 
                                        'immune_state_implanting_rate': immune_state_implanting_rate, 
                                        "protocol_implanting_rate": protocol_implanting_rate})

selection_node = ds.Selection(name="S", function=get_selection,
                              kwargs={"hospital": hospital_node, "immune_state": immune_state_node})

# make a causal graph using DagSim and show it graphically

graph = ds.Graph(name="graph_experiment_2_1", 
                 list_nodes=[immune_state_node, hospital_node, experimental_protocol_node, repertoire_node, 
                             selection_node])
graph.draw()

/home/ghadi/miniconda3/envs/CausalAIRR/lib/python3.8/subprocess.py:946: ResourceWarning:

subprocess 191899 is still running



In [73]:
training_data_sc1 = graph.simulate(num_samples=train_example_count, selection=True,
                                   csv_name=str(scenario1_data_path / "train/study_cohort"))

# make an AIRR dataset from the generated repertoires to be used for training

train_dataset = make_dataset(repertoire_paths=training_data_sc1["repertoire"], path=scenario1_data_path / 'train', 
                             dataset_name="experiment2_sc1_train", 
                             signal_names=[immune_state_signal.id, experimental_protocol_node.name])

Simulation started
Simulation finished in 295.0097 seconds


In [74]:
# make a test dataset

repertoire_node = ds.Node(name="repertoire", function=get_repertoire,
                             kwargs={"immune_state": immune_state_node,
                                        "experimental_protocol_id": experimental_protocol_node,
                                        "path": scenario1_data_path / "test", "sequence_count": sequence_count,
                                        "immune_state_signal": immune_state_signal,
                                        'immune_state_implanting_rate': immune_state_implanting_rate,
                                        "protocol_implanting_rate": protocol_implanting_rate})

graph = ds.Graph(name="graph_experiment_2_1",
                 list_nodes=[immune_state_node, hospital_node, experimental_protocol_node, repertoire_node,
                             selection_node])


test_data = graph.simulate(num_samples=test_example_count, csv_name=str(scenario1_data_path / "test/test_cohort"),
                           selection=False)

test_dataset = make_dataset(repertoire_paths=test_data["repertoire"], path=scenario1_data_path / 'test',
                            dataset_name="experiment2_sc1_test",
                            signal_names=[immune_state_signal.id, experimental_protocol_node.name])

# merge datasets (but the distinction between train and test will be kept in the ML analysis part)

dataset = make_AIRR_dataset(train_dataset, test_dataset, scenario1_data_path / 'full_dataset')

Simulation started
Simulation finished in 36.6201 seconds


### Step 2: Training an ML model

In [75]:
specs = {
    "definitions": {
        "datasets": {
            "dataset1": {
                "format": 'AIRR',
                "params": {
                    "path": str(scenario1_data_path / 'full_dataset'),
                    "metadata_file": str(scenario1_data_path / 'full_dataset/metadata.csv')
                }
            }
        },
        "encodings": {
            "kmer_frequency": {
                "KmerFrequency": {"k": 3}
            }
        },
        "ml_methods": {
            "logistic_regression": {
                "LogisticRegression": {
                    "penalty": "l1",
                    "C": [0.01, 0.1, 1, 10, 100],
                    "max_iter": 1500,
                    "show_warnings": False
                },
                "model_selection_cv": True,
                "model_selection_n_folds": 5
            }
        },
        "reports": {
            "coefficients": {
                "Coefficients": { # show top 25 logistic regression coefficients and what k-mers they correspond to
                    "coefs_to_plot": ['n_largest'],
                    "n_largest": [25]
                }
            },
            "feature_comparison": {
                "FeatureComparison": {
                    "comparison_label": "immune_state",
                    "color_grouping_label": "experimental_protocol",
                    "show_error_bar": False,
                    "keep_fraction": 0.1
                }
            }
        }
    },
    "instructions": {
        'train_ml': {
            "type": "TrainMLModel",
            "assessment": { # ensure here that train and test dataset are fixed, as per simulation
                "split_strategy": "manual",
                "split_count": 1,
                "manual_config": {
                    "train_metadata_path": str(scenario1_data_path / "train/experiment2_sc1_train_metadata.csv"),
                    "test_metadata_path": str(scenario1_data_path / "test/experiment2_sc1_test_metadata.csv")
                },
                "reports": {
                    "models": ["coefficients"],
                    "encoding": ["feature_comparison"]
                }
            },
            "selection": {
                "split_strategy": "random",
                "train_percentage": 0.7,
                "split_count": 5,
                "reports": {
                    "models": ["coefficients"],
                    "encoding": ["feature_comparison"]
                }
            },
            "settings": [
                {"encoding": "kmer_frequency", "ml_method": "logistic_regression"}
            ],
            "dataset": "dataset1",
            "refit_optimal_model": False,
            "labels": ["immune_state"],
            "optimization_metric": "balanced_accuracy",
            "metrics": ['log_loss', 'auc']
        }
    }
}

scenario1_ml_result_path = setup_path("./experiment2/scenario1/ml_result/")
scenario1_specs_path = scenario1_ml_result_path / "specs.yaml"

with open(scenario1_specs_path, "w") as file:
    yaml.dump(specs, file)

In [76]:
# run immuneML with the specs file

from immuneML.app.ImmuneMLApp import ImmuneMLApp

scenario1_output_path = scenario1_ml_result_path / "result/"

app = ImmuneMLApp(specification_path = scenario1_specs_path, result_path = scenario1_output_path)
result = app.run()

print("The results are located under ./experiment2/scenario1/")

2022-01-27 19:15:12.715675: Setting temporary cache path to experiment2/scenario1/ml_result/result/cache
2022-01-27 19:15:12.716098: ImmuneML: parsing the specification...

2022-01-27 19:15:30.936165: Full specification is available at experiment2/scenario1/ml_result/result/full_specs.yaml.

2022-01-27 19:15:30.936714: ImmuneML: starting the analysis...

2022-01-27 19:15:30.936987: Instruction 1/1 has started.
2022-01-27 19:15:30.973691: Training ML model: running outer CV loop: started split 1/1.

2022-01-27 19:15:31.017215: Hyperparameter optimization: running the inner loop of nested CV: selection for label immune_state (label 1 / 1).

2022-01-27 19:15:31.017816: Evaluating hyperparameter setting: kmer_frequency_logistic_regression...
2022-01-27 19:15:31.018498: Encoding started...
2022-01-27 19:15:35.674985: Encoding finished.
2022-01-27 19:15:35.675123: ML model training started...
2022-01-27 19:16:36.117274: ML model training finished.
2022-01-27 19:16:40.308058: Encoding started

In [77]:
from util.plotting import plot_balanced_error_rate
    
plot_balanced_error_rate(iml_result=result, result_path=scenario1_ml_result_path)

In [78]:
# show what the model has learned

from IPython.display import IFrame


IFrame(src=str(scenario1_ml_result_path / "result/HTML_output/train_ml_split_1_immune_state_kmer_frequency_logistic_regression_optimal_reports_ml_method_coefficients_largest_25_coefficients.html"),  width=700, height=600)

## Scenario 2: immune state does not influence the AIRR: learning spurious correlations

In [10]:
# define and build path, remove content if not empty

scenario2_path = setup_path("./experiment2/scenario2/")
scenario2_data_path = setup_path(scenario2_path / "data")

# define constants for the simulation

p_immune_state = 0.5 # parameter of binomial distribution for the immune state
p_hospital = 0.5 # parameter of binomial distribution for selecting between hospitals 1 and 2

sequence_count = 2000
immune_state_implanting_rate = 0.0
protocol_implanting_rate = 0.04

immune_state_signal = make_immune_state_signal()

In [11]:
immune_state_node = ds.Node(name="immune_state", function=get_immune_state, kwargs={"p": p_immune_state})

hospital_node = ds.Node(name="hospital", function=get_hospital, kwargs={"p": p_hospital})

experimental_protocol_node = ds.Node(name="exp_protocol", function=get_exp_protocol, kwargs={"hospital": hospital_node})

selection_node = ds.Selection(name="S", function=get_selection,
                              kwargs={"hospital": hospital_node, "immune_state": immune_state_node})

repertoire_node = ds.Node(name="repertoire", function=get_repertoire,
                             kwargs={"immune_state": immune_state_node, "experimental_protocol_id": experimental_protocol_node,
                                        "path": scenario2_data_path / "train", "sequence_count": sequence_count, 
                                        "immune_state_signal": immune_state_signal,
                                        'immune_state_implanting_rate': immune_state_implanting_rate,
                                        "protocol_implanting_rate": protocol_implanting_rate})

# make a causal graph using DagSim and show it graphically

graph = ds.Graph(name="graph_experiment_2_2", 
                 list_nodes=[immune_state_node, hospital_node, experimental_protocol_node,
                             repertoire_node, selection_node])
graph.draw()

/home/ghadi/miniconda3/envs/CausalAIRR/lib/python3.8/subprocess.py:946: ResourceWarning:

subprocess 73987 is still running



In [12]:
# make a train dataset

training_data_sc2 = graph.simulate(num_samples=train_example_count, 
                                   csv_name=str(scenario2_data_path / "train/study_cohort"))

train_dataset = make_dataset(repertoire_paths=training_data_sc2["repertoire"], path=scenario2_data_path / 'train',
                             dataset_name="experiment2_sc2_train",
                             signal_names=[immune_state_signal.id, experimental_protocol_node.name])

# make a test dataset

repertoire_node = ds.Node(name="repertoire", function=get_repertoire,
                             kwargs={"immune_state": immune_state_node, "experimental_protocol_id": experimental_protocol_node,
                                        "path": scenario2_data_path / "test", "sequence_count": sequence_count,
                                        "immune_state_signal": immune_state_signal,
                                        'immune_state_implanting_rate': immune_state_implanting_rate,
                                        "protocol_implanting_rate": protocol_implanting_rate})

graph = ds.Graph(name="graph_experiment_2_2",
                 list_nodes=[immune_state_node, hospital_node, experimental_protocol_node,
                             repertoire_node, selection_node])

test_data = graph.simulate(num_samples=test_example_count, csv_name=str(scenario2_data_path / "test/test_cohort"),
                           selection=False)

test_dataset = make_dataset(repertoire_paths=test_data["repertoire"], path=scenario2_data_path / 'test',
                            dataset_name="experiment2_sc2_test",
                            signal_names=[immune_state_signal.id, experimental_protocol_node.name])

# merge datasets (but the distinction between train and test will be kept in the ML analysis part)

dataset = make_AIRR_dataset(train_dataset, test_dataset, scenario2_data_path / 'full_dataset')

Simulation started


Simulation finished in 435.3755 seconds
Simulation started


Simulation finished in 54.7418 seconds


### Step 2: Training an ML model

In [13]:
specs = {
    "definitions": {
        "datasets": {
            "dataset1": {
                "format": 'AIRR',
                "params": {
                    "path": str(scenario2_data_path / 'full_dataset'),
                    "metadata_file": str(scenario2_data_path / 'full_dataset/metadata.csv')
                }
            }
        },
        "encodings": {
            "kmer_frequency": {
                "KmerFrequency": {"k": 3}
            }
        },
        "ml_methods": {
            "logistic_regression": {
                "LogisticRegression": {
                    "penalty": "l1",
                    "C": [0.01, 0.1, 1, 10, 100],
                    "max_iter": 1500,
                    "show_warnings": False
                },
                "model_selection_cv": True,
                "model_selection_n_folds": 5
            }
        },
        "reports": {
            "coefficients": {
                "Coefficients": { # show top 25 logistic regression coefficients and what k-mers they correspond to
                    "coefs_to_plot": ['n_largest'],
                    "n_largest": [25]
                }
            },
            "feature_comparison": {
                "FeatureComparison": {
                    "comparison_label": "immune_state",
                    "color_grouping_label": "experimental_protocol",
                    "show_error_bar": False,
                    "keep_fraction": 0.1
                }
            }
        }
    },
    "instructions": {
        'train_ml': {
            "type": "TrainMLModel",
            "assessment": { # ensure here that train and test dataset are fixed, as per simulation
                "split_strategy": "manual",
                "split_count": 1,
                "manual_config": {
                    "train_metadata_path": str(scenario2_data_path / "train/experiment2_sc2_train_metadata.csv"),
                    "test_metadata_path": str(scenario2_data_path / "test/experiment2_sc2_test_metadata.csv")
                },
                "reports": {
                    "models": ["coefficients"],
                    "encoding": ["feature_comparison"]
                }
            },
            "selection": {
                "split_strategy": "k_fold",
                "split_count": 5,
                "reports": {
                    "models": ["coefficients"],
                    "encoding": ["feature_comparison"]
                }
            },
            "settings": [
                {"encoding": "kmer_frequency", "ml_method": "logistic_regression"}
            ],
            "dataset": "dataset1",
            "refit_optimal_model": False,
            "labels": ["immune_state"],
            "optimization_metric": "balanced_accuracy",
            "metrics": ['log_loss', 'auc']
        }
    }
}

scenario2_ml_result_path = setup_path("./experiment2/scenario2/ml_result/")
scenario2_specs_path = scenario2_ml_result_path / "specs.yaml"

with open(scenario2_specs_path, "w") as file:
    yaml.dump(specs, file)

In [14]:
# run immuneML with the specs file

from immuneML.app.ImmuneMLApp import ImmuneMLApp

scenario2_output_path = scenario2_ml_result_path / "result/"

app = ImmuneMLApp(specification_path = scenario2_specs_path, result_path = scenario2_output_path)
result = app.run()

print("The results are located under ./experiment2/scenario2/")

2022-01-27 11:01:08.140290: Setting temporary cache path to experiment2/scenario2/ml_result/result/cache
2022-01-27 11:01:08.140737: ImmuneML: parsing the specification...

2022-01-27 11:01:32.486896: Full specification is available at experiment2/scenario2/ml_result/result/full_specs.yaml.

2022-01-27 11:01:32.487772: ImmuneML: starting the analysis...

2022-01-27 11:01:32.488321: Instruction 1/1 has started.
2022-01-27 11:01:32.555398: Training ML model: running outer CV loop: started split 1/1.

2022-01-27 11:01:32.599803: Hyperparameter optimization: running the inner loop of nested CV: selection for label immune_state (label 1 / 1).

2022-01-27 11:01:32.601086: Evaluating hyperparameter setting: kmer_frequency_logistic_regression...
2022-01-27 11:01:32.601840: Encoding started...
2022-01-27 11:01:41.402139: Encoding finished.
2022-01-27 11:01:41.402323: ML model training started...
2022-01-27 11:03:07.996020: ML model training finished.
2022-01-27 11:03:13.836352: Encoding started

In [20]:
from util.plotting import plot_balanced_error_rate
    
plot_balanced_error_rate(iml_result=result, result_path=scenario2_ml_result_path)

In [18]:
# show what the model has learned

from IPython.display import IFrame


IFrame(src=str(scenario2_ml_result_path / "result/HTML_output/train_ml_split_1_immune_state_kmer_frequency_logistic_regression_optimal_reports_ml_method_coefficients_largest_25_coefficients.html"),  width=700, height=600)